In [4]:
%cd yolov8_DeepSORT

[WinError 2] The system cannot find the file specified: 'yolov8_DeepSORT'
C:\Users\ASUS\Desktop\AU\Tracking-and-counting-Using-YOLOv8-and-DeepSORT-main


# Testing YOLOv8

In [14]:
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

# Load a model
model = YOLO("yolov8n.pt")  

results = model(r"C:\Users\ASUS\Downloads\download.jpeg", save=True)



class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

for result in results:
    boxes = result.boxes  
    probs = result.probs  
    cls = boxes.cls.tolist()  
    xyxy = boxes.xyxy
    xywh = boxes.xywh  
    conf = boxes.conf
    print(cls)
    for class_index in cls:
        class_name = class_names[int(class_index)]
        print("Class:", class_name)


image 1/1 C:\Users\ASUS\Downloads\download.jpeg: 448x640 15 persons, 1 bench, 5 backpacks, 133.3ms
Speed: 0.0ms preprocess, 133.3ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict4


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.0, 0.0, 0.0, 0.0, 0.0, 13.0, 24.0, 24.0, 24.0, 24.0]
Class: person
Class: person
Class: person
Class: person
Class: person
Class: person
Class: person
Class: person
Class: person
Class: person
Class: person
Class: backpack
Class: person
Class: person
Class: person
Class: person
Class: bench
Class: backpack
Class: backpack
Class: backpack
Class: backpack


# SINGLE VIDEO DETECTION AND TRACKING USING YOLOv8 AND DEEPSORT

In [15]:
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

In [13]:
import torch
import cv2


video_path = r"C:\Users\ASUS\Downloads\Untitled video - Made with Clipchamp (1).mp4"

cap = cv2.VideoCapture(video_path)


frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
frames = []

unique_track_ids = set()
i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        
        og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = og_frame.copy()

        model = YOLO("yolov8n.pt")  

        results = model(frame, device='cpu', classes=0, conf=0.8)

        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        for result in results:
            boxes = result.boxes  
            probs = result.probs  
            cls = boxes.cls.tolist()  
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  
            for class_index in cls:
                class_name = class_names[int(class_index)]
                

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, og_frame)
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  
            w = x2 - x1  
            h = y2 - y1  

            
            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

            text_color = (0, 0, 0) 
            cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            
            unique_track_ids.add(track_id)

       
        person_count = len(unique_track_ids)

        
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time

        
        cv2.putText(og_frame, f"Person Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        
        frames.append(og_frame)
        cv2.imshow("Video", og_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
             break

cap.release()
cv2.destroyAllWindows()



0: 384x640 2 persons, 84.4ms
Speed: 3.5ms preprocess, 84.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 98.9ms
Speed: 16.1ms preprocess, 98.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 89.7ms
Speed: 10.7ms preprocess, 89.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 102.5ms
Speed: 12.6ms preprocess, 102.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 109.6ms
Speed: 4.9ms preprocess, 109.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 111.0ms
Speed: 0.0ms preprocess, 111.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 115.7ms
Speed: 9.3ms preprocess, 115.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 100.3ms
Speed: 15.8ms preprocess, 100.3ms inference, 0.0ms postprocess per im

# TWO CAMERA PERSON DETECTION AND TRACKING USING DEEPSORT

In [49]:
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker
from ultralytics import YOLO
import numpy as np
import cv2
import torch



# Define the video paths
video_path1 = r"C:\Users\ASUS\Downloads\WhatsApp Video 2023-10-03 at 00.28.41_76315df1.mp4"
video_path2 = r"C:\Users\ASUS\Downloads\WhatsApp Video 2023-10-03 at 00.28.42_4b733403.mp4"


deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker1 = DeepSort(model_path=deep_sort_weights, max_age=70)
tracker2 = DeepSort(model_path=deep_sort_weights, max_age=70)

# Create video capture objects
cap1 = cv2.VideoCapture(video_path1)
cap2 = cv2.VideoCapture(video_path2)

# Define the frame buffers
frames1 = []
frames2 = []

# Start the loop
while cap1.isOpened() and cap2.isOpened():
    # Read frames from the two cameras
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    # If a frame is not read from one of the cameras, break the loop
    if not ret1 or not ret2:
        break

    og_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
    frame1 = og_frame1.copy()
    og_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    frame2 = og_frame2.copy()

    # Perform object detection and tracking on each camera
    model = YOLO("yolov8n.pt")

    # Object detection on camera 1
    results1 = model(frame1, device='cpu', classes=0, conf=0.8)

    # Object detection on camera 2
    results2 = model(frame2, device='cpu', classes=0, conf=0.8)

    class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

    # Tracking on camera 1
    for result in results1:
            boxes = result.boxes  
            probs = result.probs  
            cls = boxes.cls.tolist()  
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  
            for class_index in cls:
                class_name = class_names[int(class_index)]
                

    pred_cls = np.array(cls)
    conf = conf.detach().cpu().numpy()
    xyxy = xyxy.detach().cpu().numpy()
    bboxes_xywh1 = xywh
    bboxes_xywh1 = xywh.cpu().numpy()
    bboxes_xywh1= np.array(bboxes_xywh1, dtype=float)
     # Assuming 'results1' is a list with detection results
     # Assuming 'results1' is a list with detection results
    tracks1 = tracker1.update(bboxes_xywh1, conf, og_frame1)

    # Tracking on camera 2
    for result in results2:
            boxes = result.boxes  
            probs = result.probs  
            cls = boxes.cls.tolist()  
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  
            for class_index in cls:
                class_name = class_names[int(class_index)]
                

    pred_cls = np.array(cls)
    conf = conf.detach().cpu().numpy()
    xyxy = xyxy.detach().cpu().numpy()
    bboxes_xywh2 = xywh
    bboxes_xywh2 = xywh.cpu().numpy()
    bboxes_xywh2= np.array(bboxes_xywh2, dtype=float)
     # Assuming 'results2' is a list with detection results
    tracks2 = tracker2.update(bboxes_xywh2, conf, og_frame2)

    # Merge the tracks from both cameras
    #tracks = tracks1 + tracks2
    #tracks=tracks1

    # Draw the bounding boxes and track IDs on the frames
    for track in tracker1.tracker.tracks:
        track_id = track.track_id
        hits = track.hits
        x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
        w = x2 - x1  # Calculate width
        h = y2 - y1  # Calculate height

        # Set color values for red, blue, and green
        red_color = (0, 0, 255)  # (B, G, R)
        blue_color = (255, 0, 0)  # (B, G, R)
        green_color = (0, 255, 0)  # (B, G, R)

        # Determine color based on track_id
        color = green_color

        cv2.rectangle(og_frame1, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(og_frame1, f"{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 1, cv2.LINE_AA)
        #cv2.rectangle(og_frame2, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        #cv2.putText(og_frame2, f"{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    #0.5, color, 1, cv2.LINE_AA)
    #tracks=tracks2
    for track in tracker2.tracker.tracks:
        track_id = track.track_id
        hits = track.hits
        x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
        w = x2 - x1  # Calculate width
        h = y2 - y1  # Calculate height

        # Set color values for red, blue, and green
        red_color = (0, 0, 255)  # (B, G, R)
        blue_color = (255, 0, 0)  # (B, G, R)
        green_color = (0, 255, 0)  # (B, G, R)

        # Determine color based on track_id
        color = green_color
        cv2.rectangle(og_frame2, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(og_frame2, f"{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 1, cv2.LINE_AA)

    # Append the frame to the frame buffers
    #frames1.append(frame1)
    #frames2.append(frame2)

    cv2.imshow("Video1", og_frame1)
    cv2.imshow("Video2", og_frame2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture objects
cap1.release()
cap2.release()

# Destroy all windows
cv2.destroyAllWindows()


0: 384x640 (no detections), 101.7ms
Speed: 4.4ms preprocess, 101.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 107.7ms
Speed: 0.0ms preprocess, 107.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 100.2ms
Speed: 3.8ms preprocess, 100.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 85.1ms
Speed: 4.6ms preprocess, 85.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 116.8ms
Speed: 0.0ms preprocess, 116.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 88.3ms
Speed: 0.0ms preprocess, 88.3ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 89.3ms
Speed: 8.5ms preprocess, 89.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 87.7ms
Speed: 0.0ms preprocess, 87.7ms inference, 0.0ms post